## WEB scraping info

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
url = 'https://en.wikipedia.org/wiki/Berlin'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [5]:
 def city_info(city_list):
    city_info = []
    for city in city_list:
      url = f'https://en.wikipedia.org/wiki/{city}'
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      country = soup.find(string='Country').find_next().get_text()
      longitude = soup.find(string='Coordinates: ').find_next().get_text().split()[-1].strip(';')
      latitude = soup.find(string='Coordinates: ').find_next().get_text().split()[-2].strip(';')
      metro_pop= int(soup.find(title='Metropolitan area').find_next('td').get_text(separator=' ').split()[0].replace(',', ''))
      city_area = float(repr(soup.find(string='Area').find_next('td').get_text()).split('\\')[0].strip("'"))
      city_info.append([city, country, latitude, longitude, metro_pop, city_area])
    return pd.DataFrame(city_info, columns=['name', 'country', 'latitude', 'longitude', 'metro_pop', 'city_area'])

cities = ('Berlin', 'Hamburg', 'Munich')
cities_df = city_info(cities)
cities_df

,name,country,latitude,longitude,metro_pop,city_area
0,Berlin,Germany,52.52000,13.40500,6144600,891.30
1,Hamburg,Germany,53.550,10.000,5425628,755.22
2,Munich,Germany,48.13750,11.57500,5991144,310.71


In [6]:
def city_info(city_list):
    city_info = []
    for city in city_list:
      url = f'https://en.wikipedia.org/wiki/{city}'
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      country = soup.find(string='Country').find_next().get_text()
      longitude = soup.find(string='Coordinates: ').find_next().get_text().split()[-1].strip(';')
      latitude = soup.find(string='Coordinates: ').find_next().get_text().split()[-2].strip(';')
      city_info.append([city, country, latitude, longitude])
    return pd.DataFrame(city_info, columns=['city', 'country', 'latitude', 'longitude'])

cities = ('Berlin', 'Hamburg', 'Munich')
cities_stable_df = city_info(cities)
cities_stable_df

,city,country,latitude,longitude
0,Berlin,Germany,52.52000,13.40500
1,Hamburg,Germany,53.550,10.000
2,Munich,Germany,48.13750,11.57500


In [7]:
 def city_info(city_list):
    city_info = []
    for city in city_list:
      url = f'https://en.wikipedia.org/wiki/{city}'
      response = requests.get(url)
      soup = BeautifulSoup(response.content, 'html.parser')
      country = soup.find(string='Country').find_next().get_text()
      metro_pop= int(soup.find(title='Metropolitan area').find_next('td').get_text(separator=' ').split()[0].replace(',', ''))
      city_area = float(repr(soup.find(string='Area').find_next('td').get_text()).split('\\')[0].strip("'"))
      city_info.append([city, metro_pop, city_area])
    return pd.DataFrame(city_info, columns=['city', 'metro_pop', 'city_area'])

cities = ('Berlin', 'Hamburg', 'Munich')
cities_unstable_df = city_info(cities)

cities_unstable_df['collected'] = pd.Timestamp.now().date()

cities_unstable_df

,city,metro_pop,city_area,collected
0,Berlin,6144600,891.30,2024-08-12
1,Hamburg,5425628,755.22,2024-08-12
2,Munich,5991144,310.71,2024-08-12


In [8]:
cities_unstable_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   city       3 non-null      object 
 1   metro_pop  3 non-null      int64  
 2   city_area  3 non-null      float64
 3   collected  3 non-null      object 
dtypes: float64(1), int64(1), object(2)
memory usage: 228.0+ bytes


## WEB scraping data tables transfer to SQL 

In [10]:
schema = "database_cities"
host = "127.0.0.1"
user = "root"
password = "Jasiek1994!"
port = 3306

connection_string = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [11]:
#cities_stable_df.to_sql('cities_stable',
                  #if_exists='append',
                  #con=connection_string,
                  #index=False)

In [12]:
cities_stable_from_sql = pd.read_sql("cities_stable", con=connection_string)
cities_stable_from_sql

,city_id,city,country,latitude,longitude
0,1,Berlin,Germany,52.5200,13.405
1,2,Hamburg,Germany,53.5500,10.000
2,3,Munich,Germany,48.1375,11.575


In [13]:
cities_unstable_df = cities_unstable_df.merge(cities_stable_from_sql,
                                              on = "city",
                                              how="left")

cities_unstable_df

,city,metro_pop,city_area,collected,city_id,country,latitude,longitude
0,Berlin,6144600,891.30,2024-08-12,1,Germany,52.5200,13.405
1,Hamburg,5425628,755.22,2024-08-12,2,Germany,53.5500,10.000
2,Munich,5991144,310.71,2024-08-12,3,Germany,48.1375,11.575


In [14]:
cities_unstable_df = cities_unstable_df.drop(columns=['city', 'country', 'latitude', 'longitude'])

cities_unstable_df

,metro_pop,city_area,collected,city_id
0,6144600,891.30,2024-08-12,1
1,5425628,755.22,2024-08-12,2
2,5991144,310.71,2024-08-12,3


In [15]:
#cities_unstable_df.to_sql('cities_unstable',
                  #if_exists='append',
                  #con=connection_string,
                  #index=False)

## JSON - Weather for all cities 

In [17]:
import sys
import json
import requests

In [18]:
latitude = 52.52
longitude = 13.24
date = '2024-08-08'
API_key = '7dbecbe77d3101cfa848a85bded60217'


In [19]:
cities_stable_from_sql.loc[0]

city_id            1
city          Berlin
country      Germany
latitude       52.52
longitude     13.405
Name: 0, dtype: object

In [20]:
berlin = cities_stable_from_sql.loc[0]

In [21]:
response_berlin = requests.get(f'http://api.openweathermap.org/data/2.5/forecast?lat={berlin['latitude']}&lon={berlin['longitude']}&appid={API_key}&units=metric')

In [22]:
berlin_json = response_berlin.json()

In [23]:
berlin_json['list'][0]

{'dt': 1723464000,
 'main': {'temp': 27.05,
  'feels_like': 27.17,
  'temp_min': 27.05,
  'temp_max': 27.81,
  'pressure': 1015,
  'sea_level': 1015,
  'grnd_level': 1010,
  'humidity': 45,
  'temp_kf': -0.76},
 'weather': [{'id': 800,
   'main': 'Clear',
   'description': 'clear sky',
   'icon': '01d'}],
 'clouds': {'all': 0},
 'wind': {'speed': 0.56, 'deg': 344, 'gust': 1.23},
 'visibility': 10000,
 'pop': 0,
 'sys': {'pod': 'd'},
 'dt_txt': '2024-08-12 12:00:00'}

In [24]:
berlin_json['list'][0]['main']['temp_min']
berlin_json['list'][0]['main']['temp_max']
berlin_json['list'][0]['main']['feels_like']
berlin_json['list'][0]['main']['humidity']
berlin_json['list'][0]['main']['pressure']
berlin_json['list'][0]['weather'][0]['main']
berlin_json['list'][0]['wind']['speed']
berlin_json['list'][0]['visibility']
berlin_json['list'][0].get('rain', {}).get('3h')
berlin_json['list'][0]['dt_txt']

'2024-08-12 12:00:00'

In [25]:
temp_min = []
temp_max = []
feels_like = []
humidity = []
pressure = []
main = []
speed = []
visibility = []
rain = []
time = []
city_id = []

for forecast in berlin_json['list']:
    temp_min.append(forecast['main']['temp_min'])
    temp_max.append(forecast['main']['temp_max'])
    feels_like.append(forecast['main']['feels_like'])
    humidity.append(forecast['main']['humidity'])
    pressure.append(forecast['main']['pressure'])
    main.append(forecast['weather'][0]['main'])
    speed.append(forecast['wind']['speed'])
    visibility.append(forecast['visibility'])
    rain.append(forecast.get('rain', {}).get('3h'))
    time.append(forecast['dt_txt'])
    city_id.append(berlin['city_id'])
   



In [26]:
 pd.DataFrame ({'temp_min': temp_min,
                'temp_max': temp_max,  
                'feels_like': feels_like,   
                'humidity': humidity,  
                'pressure': pressure,  
                'main': main, 
                'speed': speed,  
                'visibility': visibility, 
                'rain': rain,  
                'time': time,  
                'city_id': city_id })

,temp_min,temp_max,feels_like,humidity,pressure,main,speed,visibility,rain,time,city_id
0,27.05,27.81,27.17,45,1015,Clear,0.56,10000,NaN,2024-08-12 12:00:00,1
1,27.43,28.20,27.13,39,1015,Clouds,1.36,10000,NaN,2024-08-12 15:00:00,1
2,25.30,25.88,25.57,40,1014,Clouds,2.67,10000,NaN,2024-08-12 18:00:00,1
3,21.55,21.55,20.99,47,1014,Clouds,3.53,10000,NaN,2024-08-12 21:00:00,1
4,19.10,19.10,18.48,54,1015,Clouds,3.39,10000,NaN,2024-08-13 00:00:00,1
5,17.62,17.62,17.00,60,1014,Clear,3.24,10000,NaN,2024-08-13 03:00:00,1
6,19.34,19.34,18.74,54,1014,Clear,3.77,10000,NaN,2024-08-13 06:00:00,1
7,25.31,25.31,24.89,38,1014,Clear,3.87,10000,NaN,2024-08-13 09:00:00,1
8,29.74,29.74,28.39,28,1013,Clear,3.84,10000,NaN,2024-08-13 12:00:00,1
9,30.90,30.90,29.25,25,1012,Clear,3.75,10000,NaN,2024-08-13 15:00:00,1


In [27]:
temp_min = []
temp_max = []
feels_like = []
humidity = []
pressure = []
main = []
speed = []
visibility = []
rain = []
time = []
city_id = []


for i,row in cities_stable_from_sql.iterrows():
    response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast?lat={row['latitude']}&lon={row['longitude']}&appid={API_key}&units=metric')
    response_json = response.json()
    if response.status_code == 200:
        for forecast in response_json['list']:
            temp_min.append(forecast['main']['temp_min'])
            temp_max.append(forecast['main']['temp_max'])
            feels_like.append(forecast['main']['feels_like'])
            humidity.append(forecast['main']['humidity'])
            pressure.append(forecast['main']['pressure'])
            main.append(forecast['weather'][0]['main'])
            speed.append(forecast['wind']['speed'])
            visibility.append(forecast['visibility'])
            rain.append(forecast.get('rain', {}).get('3h'))
            time.append(forecast['dt_txt'])
            city_id.append(row['city_id'])
    
weather_df = pd.DataFrame ({'temp_min': temp_min,
                'temp_max': temp_max,  
                'feels_like': feels_like,   
                'humidity': humidity,  
                'pressure': pressure,  
                'main': main, 
                'speed': speed,  
                'visibility': visibility, 
                'rain': rain,  
                'time': time,  
                'city_id': city_id })

weather_df

,temp_min,temp_max,feels_like,humidity,pressure,main,speed,visibility,rain,time,city_id
0,27.05,27.81,27.17,45,1015,Clear,0.56,10000,NaN,2024-08-12 12:00:00,1
1,27.43,28.20,27.13,39,1015,Clouds,1.36,10000,NaN,2024-08-12 15:00:00,1
2,25.30,25.88,25.57,40,1014,Clouds,2.67,10000,NaN,2024-08-12 18:00:00,1
3,21.55,21.55,20.99,47,1014,Clouds,3.53,10000,NaN,2024-08-12 21:00:00,1
4,19.10,19.10,18.48,54,1015,Clouds,3.39,10000,NaN,2024-08-13 00:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...
115,19.48,19.48,20.02,97,1016,Rain,4.54,10000,20.13,2024-08-16 21:00:00,3
116,18.13,18.13,18.51,96,1016,Rain,1.74,10000,0.36,2024-08-17 00:00:00,3
117,17.63,17.63,17.90,94,1016,Rain,1.50,10000,0.15,2024-08-17 03:00:00,3
118,18.31,18.31,18.60,92,1016,Clouds,0.59,10000,NaN,2024-08-17 06:00:00,3


In [28]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   temp_min    120 non-null    float64
 1   temp_max    120 non-null    float64
 2   feels_like  120 non-null    float64
 3   humidity    120 non-null    int64  
 4   pressure    120 non-null    int64  
 5   main        120 non-null    object 
 6   speed       120 non-null    float64
 7   visibility  120 non-null    int64  
 8   rain        31 non-null     float64
 9   time        120 non-null    object 
 10  city_id     120 non-null    int64  
dtypes: float64(5), int64(4), object(2)
memory usage: 10.4+ KB


In [29]:
weather_df['time'] = pd.to_datetime(weather_df['time'], utc=True).dt.tz_convert('Europe/Berlin')
weather_df.head()

,temp_min,temp_max,feels_like,humidity,pressure,main,speed,visibility,rain,time,city_id
0,27.05,27.81,27.17,45,1015,Clear,0.56,10000,NaN,2024-08-12 14:00:00+02:00,1
1,27.43,28.20,27.13,39,1015,Clouds,1.36,10000,NaN,2024-08-12 17:00:00+02:00,1
2,25.30,25.88,25.57,40,1014,Clouds,2.67,10000,NaN,2024-08-12 20:00:00+02:00,1
3,21.55,21.55,20.99,47,1014,Clouds,3.53,10000,NaN,2024-08-12 23:00:00+02:00,1
4,19.10,19.10,18.48,54,1015,Clouds,3.39,10000,NaN,2024-08-13 02:00:00+02:00,1


In [30]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype                        
---  ------      --------------  -----                        
 0   temp_min    120 non-null    float64                      
 1   temp_max    120 non-null    float64                      
 2   feels_like  120 non-null    float64                      
 3   humidity    120 non-null    int64                        
 4   pressure    120 non-null    int64                        
 5   main        120 non-null    object                       
 6   speed       120 non-null    float64                      
 7   visibility  120 non-null    int64                        
 8   rain        31 non-null     float64                      
 9   time        120 non-null    datetime64[ns, Europe/Berlin]
 10  city_id     120 non-null    int64                        
dtypes: datetime64[ns, Europe/Berlin](1), float64(5), int64(4), object(1)
me

In [31]:
def weather_data(cities_stable_from_sql):
    temp_min = []
    temp_max = []
    feels_like = []
    humidity = []
    pressure = []
    main = []
    speed = []
    visibility = []
    rain = []
    time = []
    city_id = []
    for i,row in cities_stable_from_sql.iterrows():
        response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast?lat={row['latitude']}&lon={row['longitude']}&appid={API_key}&units=metric')
        response_json = response.json()
        if response.status_code == 200:
            for forecast in response_json['list']:
                temp_min.append(forecast['main']['temp_min'])
                temp_max.append(forecast['main']['temp_max'])
                feels_like.append(forecast['main']['feels_like'])
                humidity.append(forecast['main']['humidity'])
                pressure.append(forecast['main']['pressure'])
                main.append(forecast['weather'][0]['main'])
                speed.append(forecast['wind']['speed'])
                visibility.append(forecast['visibility'])
                rain.append(forecast.get('rain', {}).get('3h'))
                time.append(forecast['dt_txt'])
                city_id.append(row['city_id'])
    return pd.DataFrame ({'temp_min': temp_min,
                'temp_max': temp_max,  
                'feels_like': feels_like,   
                'humidity': humidity,  
                'pressure': pressure,  
                'main': main, 
                'speed': speed,  
                'visibility': visibility, 
                'rain': rain,  
                'time': time,  
                'city_id': city_id })

weather_df = weather_data(cities_stable_from_sql)
weather_df

,temp_min,temp_max,feels_like,humidity,pressure,main,speed,visibility,rain,time,city_id
0,27.05,27.81,27.17,45,1015,Clear,0.56,10000,NaN,2024-08-12 12:00:00,1
1,27.43,28.20,27.13,39,1015,Clouds,1.36,10000,NaN,2024-08-12 15:00:00,1
2,25.30,25.88,25.57,40,1014,Clouds,2.67,10000,NaN,2024-08-12 18:00:00,1
3,21.55,21.55,20.99,47,1014,Clouds,3.53,10000,NaN,2024-08-12 21:00:00,1
4,19.10,19.10,18.48,54,1015,Clouds,3.39,10000,NaN,2024-08-13 00:00:00,1
...,...,...,...,...,...,...,...,...,...,...,...
115,19.48,19.48,20.02,97,1016,Rain,4.54,10000,20.13,2024-08-16 21:00:00,3
116,18.13,18.13,18.51,96,1016,Rain,1.74,10000,0.36,2024-08-17 00:00:00,3
117,17.63,17.63,17.90,94,1016,Rain,1.50,10000,0.15,2024-08-17 03:00:00,3
118,18.31,18.31,18.60,92,1016,Clouds,0.59,10000,NaN,2024-08-17 06:00:00,3


## Weather_df to.sql()

In [33]:
#weather_df.to_sql('weather_info',
                  #if_exists='append',
                  #con=connection_string,
                  #index=False)

In [34]:
weather_from_sql = pd.read_sql("weather_info", con=connection_string)
weather_from_sql

,weather_id,temp_min,temp_max,feels_like,humidity,pressure,main,speed,visibility,rain,time,city_id
0,1,16.01,18.18,15.81,82,1020,Clear,1.95,10000,NaN,2024-08-12,1
1,2,16.37,17.09,16.05,76,1020,Clear,1.54,10000,NaN,2024-08-12,1
2,3,17.79,18.68,17.35,66,1019,Clear,1.42,10000,NaN,2024-08-12,1
3,4,23.79,23.79,23.40,45,1017,Clear,1.03,10000,NaN,2024-08-12,1
4,5,27.64,27.64,26.93,32,1016,Clouds,0.34,10000,NaN,2024-08-12,1
...,...,...,...,...,...,...,...,...,...,...,...,...
115,116,25.28,25.28,25.64,68,1016,Clouds,0.84,10000,NaN,2024-08-16,3
116,117,28.31,28.31,29.44,56,1015,Clouds,1.69,10000,NaN,2024-08-16,3
117,118,28.56,28.56,29.55,54,1014,Clouds,3.02,10000,NaN,2024-08-16,3
118,119,21.56,21.56,22.15,91,1014,Rain,2.44,3602,5.09,2024-08-16,3


## Cities_add_info_tables

In [725]:
import re

In [726]:
url = 'https://en.wikipedia.org/wiki/Berlin'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
catholics_pop = int(soup.find(string=" (9%)").split('(')[1].split(')')[0].strip("'").split('%')[0])
protestants_pop = int(soup.find(string=re.compile("15%", re.IGNORECASE)).get_text().split('(')[1].strip("'").split(')')[0].strip("'").split('%')[0])
muslim_pop = int(soup.find(string=" (4%)").split('(')[1].split(')')[0].strip("'").split('%')[0])
not_rel_pop = int(soup.find(string=re.compile("not religious", re.IGNORECASE)).get_text().split('\xa0')[1].strip("'").split('(')[1].split(')')[0].strip("'").split('%')[0])
turks_pop = int(soup.find(title='Turkey').find_next('td').get_text().split('\n')[0].strip("'").replace(',', ''))
polish_pop = int(soup.find(title='Poland').find_next('td').get_text().split('\n')[0].strip("'").replace('.', ''))
italians_pop = int(soup.find(title='Italy').find_next('td').get_text().split('\n')[0].strip("'").replace(',', ''))
Berlin_info = ['Berlin', catholics_pop,  protestants_pop,  muslim_pop,  not_rel_pop,  turks_pop, polish_pop, italians_pop]
Berlin_info

['Berlin', 9, 15, 4, 72, 107022, 54099, 32952]

In [727]:
url = 'https://en.wikipedia.org/wiki/Hamburg'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
catholics_pop = float(soup.find(string=re.compile("9.9%", re.IGNORECASE)).get_text().split('%')[0])
protestants_pop = float(soup.find(string=re.compile("24.9%", re.IGNORECASE)).get_text().split('%')[0])
muslim_pop = float(soup.find(string=re.compile("8.4%", re.IGNORECASE)).get_text().split(' ')[-5].split('%')[0])
not_rel_pop = float(soup.find(string=re.compile("65.2%", re.IGNORECASE)).get_text().split('%')[0])
turks_pop = int(soup.find(title='Turkey').find_next('td').get_text().split('\n')[0].strip("'").replace(',', ''))
polish_pop = int(soup.find(title='Poland').find_next('td').get_text().split('\n')[0].strip("'").replace(',', ''))
italians_pop = int(soup.find(title='Italy').find_next('td').get_text().split('\n')[0].strip("'").replace(',', ''))
Hamburg_info = ['Hamburg', catholics_pop,  protestants_pop,  muslim_pop,  not_rel_pop,  turks_pop, polish_pop, italians_pop]
Hamburg_info

['Hamburg', 9.9, 24.9, 8.4, 65.2, 44280, 23310, 7570]

In [728]:
url = 'https://en.wikipedia.org/wiki/Munich'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
catholics_pop = float(soup.find(string=re.compile("31.8%", re.IGNORECASE)).get_text().split('2017,')[1].split('of')[0].split('%')[0])
protestants_pop = float(soup.find(string=re.compile(" 11.4%", re.IGNORECASE)).get_text().split(',')[1].strip("'").split('%')[0])
muslim_pop = float(soup.find(string=re.compile("8.6%", re.IGNORECASE)).get_text().split(' ')[-6].split('%')[0])
not_rel_pop = int(soup.find(string=re.compile("45%", re.IGNORECASE)).get_text().split(' ')[1].split('%')[0])
turks_pop = int(soup.find(title='Turkey').find_next('td').get_text().split('\n')[0].strip("'").replace(',', ''))
polish_pop = int(soup.find(title='Poland').find_next('td').get_text().split('\n')[0].strip("'").replace(',', ''))
italians_pop = int(soup.find(title='Italy').find_next('td').get_text().split('\n')[0].strip("'").replace(',', ''))
Munich_info = ['Munich', catholics_pop,  protestants_pop,  muslim_pop,  not_rel_pop,  turks_pop, polish_pop, italians_pop]
Munich_info

['Munich', 31.8, 11.4, 8.6, 45, 38947, 16729, 28142]

In [729]:
cities_add_info_df = pd.DataFrame([Berlin_info, Hamburg_info, Munich_info],columns=['city', 'Catholics_pop', 'Protestants_pop','Muslims_pop','Non_religious_pop', 'Turks_pop','Polish_pop','Italians_pop'])

cities_add_info_df['collected'] = pd.Timestamp.now().date()
cities_add_info_df

,city,Catholics_pop,Protestants_pop,Muslims_pop,Non_religious_pop,Turks_pop,Polish_pop,Italians_pop,collected
0,Berlin,9.0,15.0,4.0,72.0,107022,54099,32952,2024-08-13
1,Hamburg,9.9,24.9,8.4,65.2,44280,23310,7570,2024-08-13
2,Munich,31.8,11.4,8.6,45.0,38947,16729,28142,2024-08-13


In [730]:
data = {'city': ['Berlin', 'Hamburg', 'Munich'],
        'Catholics_pop(%)': [9.0, 9.9, 31.8],
        'Protestants_pop(%)': [15.0, 24.9, 11.4],
        'Muslims_pop(%)' : [4.0, 8.4, 8.6],
        'Non_religious_pop(%)': [72.0, 65.2, 45.0],
        'Turks_pop(abs)' : [107022, 44280, 38947],
        'Turks_pop(abs)' : [54099, 23310, 16729],
        'Turks_pop(abs)' : [32952, 7570, 28142],
       }

cities_add_info_df = pd.DataFrame.from_dict(data)

cities_add_info_df['collected'] = pd.Timestamp.now().date()
cities_add_info_df['collected'] = pd.to_datetime(cities_add_info_df['collected'])
cities_add_info_df

,city,Catholics_pop(%),Protestants_pop(%),Muslims_pop(%),Non_religious_pop(%),Turks_pop(abs),collected
0,Berlin,9.0,15.0,4.0,72.0,32952,2024-08-13
1,Hamburg,9.9,24.9,8.4,65.2,7570,2024-08-13
2,Munich,31.8,11.4,8.6,45.0,28142,2024-08-13


In [821]:
def cities_info():
    
    data = {'city': ['Berlin', 'Hamburg', 'Munich'],
        'Catholics_pop': [9.0, 9.9, 31.8],
        'Protestants_pop': [15.0, 24.9, 11.4],
        'Muslims_pop' : [4.0, 8.4, 8.6],
        'Non_religious_pop': [72.0, 65.2, 45.0],
        'Turks_pop' : [107022, 44280, 38947],
        'Polish_pop' : [54099, 23310, 16729],
        'Italians_pop' : [32952, 7570, 28142],
       }

    cities_add_info_df = pd.DataFrame.from_dict(data)
    
    cities_add_info_df['collected'] = pd.Timestamp.now().date()
    cities_add_info_df['collected'] = pd.to_datetime(cities_add_info_df['collected'])
    
    return cities_add_info_df

cities_add_info_df = cities_info()
cities_add_info_df

,city,Catholics_pop,Protestants_pop,Muslims_pop,Non_religious_pop,Turks_pop,Polish_pop,Italians_pop,collected
0,Berlin,9.0,15.0,4.0,72.0,107022,54099,32952,2024-08-13
1,Hamburg,9.9,24.9,8.4,65.2,44280,23310,7570,2024-08-13
2,Munich,31.8,11.4,8.6,45.0,38947,16729,28142,2024-08-13


In [823]:
cities_add_info_df = cities_add_info_df.merge(cities_stable_from_sql,
                                             on='city')
cities_add_info_df

,city,Catholics_pop,Protestants_pop,Muslims_pop,Non_religious_pop,Turks_pop,Polish_pop,Italians_pop,collected,city_id,country,latitude,longitude
0,Berlin,9.0,15.0,4.0,72.0,107022,54099,32952,2024-08-13,1,Germany,52.5200,13.405
1,Hamburg,9.9,24.9,8.4,65.2,44280,23310,7570,2024-08-13,2,Germany,53.5500,10.000
2,Munich,31.8,11.4,8.6,45.0,38947,16729,28142,2024-08-13,3,Germany,48.1375,11.575


In [825]:
cities_add_info_df = cities_add_info_df.drop(columns=['city', 'country', 'latitude', 'longitude'])
cities_add_info_df

,Catholics_pop,Protestants_pop,Muslims_pop,Non_religious_pop,Turks_pop,Polish_pop,Italians_pop,collected,city_id
0,9.0,15.0,4.0,72.0,107022,54099,32952,2024-08-13,1
1,9.9,24.9,8.4,65.2,44280,23310,7570,2024-08-13,2
2,31.8,11.4,8.6,45.0,38947,16729,28142,2024-08-13,3


## Cities_add_info to.sql()

In [827]:
#cities_add_info_df.to_sql('cities_add_info',
                          #if_exists='append',
                      #con=connection_string,
                      #index=False)

3

In [829]:
cities_add_info_from_sql = pd.read_sql("cities_add_info", con=connection_string)
cities_add_info_from_sql

,Catholics_pop,Protestants_pop,Muslims_pop,Non_religious_pop,Turks_pop,Polish_pop,Italians_pop,collected,city_id
0,9.0,15.0,4.0,72.0,107022,54099,32952,2024-08-13,1
1,9.9,24.9,8.4,65.2,44280,23310,7570,2024-08-13,2
2,31.8,11.4,8.6,45.0,38947,16729,28142,2024-08-13,3


## Weather_add_info

In [ ]:
import re

### Berlin

In [879]:
url = 'https://en.wikipedia.org/wiki/Berlin'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [880]:
jan = soup.find_all(scope='col')[:12][0].get_text().split('\n')[0].strip("'")
feb = soup.find_all(scope='col')[:12][1].get_text().split('\n')[0].strip("'")
march = soup.find_all(scope='col')[:12][2].get_text().split('\n')[0].strip("'")
april = soup.find_all(scope='col')[:12][3].get_text().split('\n')[0].strip("'")
may = soup.find_all(scope='col')[:12][4].get_text().split('\n')[0].strip("'")
jun = soup.find_all(scope='col')[:12][5].get_text().split('\n')[0].strip("'")
jul = soup.find_all(scope='col')[:12][5].get_text().split('\n')[0].strip("'")
aug = soup.find_all(scope='col')[:12][7].get_text().split('\n')[0].strip("'")
sep = soup.find_all(scope='col')[:12][8].get_text().split('\n')[0].strip("'")
oct = soup.find_all(scope='col')[:12][9].get_text().split('\n')[0].strip("'")
nov = soup.find_all(scope='col')[:12][10].get_text().split('\n')[0].strip("'")
dec = soup.find_all(scope='col')[:12][11].get_text().split('\n')[0].strip("'")

In [881]:
months = [jan, feb,march,april,may, jun,jul,aug,sep,oct,nov,dec]
months

['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jun',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec']

In [882]:
Record_high = soup.find_all(scope='row')[30].get_text().split('\n')[0].strip("'").split('(°F)')[0].strip("'")
Mean_maximum =  soup.find_all(scope='row')[31].get_text().split('\n')[0].strip("'").split('(°F)')[0].strip("'")
Mean_minimum = soup.find_all(scope='row')[35].get_text().split('\n')[0].strip("'").split('(°F)')[0].strip("'")
Record_low = soup.find_all(scope='row')[36].get_text().split('\n')[0].strip("'").split('(°F)')[0].strip("'")
Average_precipitation_mm = soup.find_all(scope='row')[37].get_text().split('\n')[0].strip("'").split('(inches)')[0].strip("'")
Average_precipitation_days = soup.find_all(scope='row')[38].get_text().split('\n')[0].strip("'").split('(°F)')[0].strip("'")
Average_snowy_days = soup.find_all(scope='row')[39].get_text().split('\n')[0].strip("'").split('(°F)')[0].strip("'")
Average_relative_humidity = soup.find_all(scope='row')[40].get_text().split('\n')[0].strip("'").split('(°F)')[0].strip("'")

In [883]:
climate_data = [Record_high, 
                Mean_maximum,
                Mean_minimum ,
                Record_low ,
                Average_precipitation_mm ,
                Average_precipitation_days ,
                Average_snowy_days ,
                Average_relative_humidity ]
climate_data

['Record high °C ',
 'Mean maximum °C ',
 'Mean minimum °C ',
 'Record low °C ',
 'Average precipitation mm ',
 'Average precipitation days (≥ 0.1 mm)',
 'Average snowy days (≥ 1.0 cm)',
 'Average relative humidity (%)']

In [884]:
soup.find_all(scope='row')[37].get_text().split('\n')[0].strip("'").split('(inches)')[0].strip("'")

'Average precipitation mm '

In [885]:
float(soup.find_all(scope='row')[30].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

15.1

In [886]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

19.2

In [887]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

25.8

In [888]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

30.8

In [889]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

32.7

In [890]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

38.4

In [891]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

38.3

In [892]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

38.0

In [893]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

32.3

In [894]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

27.7

In [895]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

20.9

In [896]:
float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

15.6

In [897]:
record_high_jan = float(soup.find_all(scope='row')[30].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_feb = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_march = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_april = float(soup.find_all(scope='row')[30].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_may = float(soup.find_all(scope='row')[30].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_jun = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_jul = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_aug = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_sep = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_oct = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_nov = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_dec = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [898]:
record_high = [record_high_jan,record_high_feb,record_high_march,record_high_april,record_high_may,record_high_jun,record_high_jul,record_high_aug,record_high_sep,
              record_high_oct,record_high_nov, record_high_dec ]
record_high

[15.1, 19.2, 25.8, 30.8, 32.7, 38.4, 38.3, 38.0, 32.3, 27.7, 20.9, 15.6]

In [899]:
mean_max_jan = float(soup.find_all(scope='row')[31].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_feb = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_march = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_april = float(soup.find_all(scope='row')[31].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_may = float(soup.find_all(scope='row')[31].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_jun = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_jul = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_aug = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_sep = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_oct = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_nov = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
mean_max_dec = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [900]:
mean_max = [mean_max_jan,mean_max_feb,mean_max_march,mean_max_april,mean_max_may,mean_max_jun,mean_max_jul,mean_max_aug,mean_max_sep,
              mean_max_oct,mean_max_nov, mean_max_dec ]

mean_max

[10.6, 12.4, 17.9, 24.0, 28.4, 31.5, 32.7, 32.7, 26.9, 21.5, 14.8, 11.2]

In [901]:
mean_min_jan = float(soup.find_all(scope='row')[35].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_feb = float(soup.find_all(scope='row')[35].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_march = float(soup.find_all(scope='row')[35].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_april = float(soup.find_all(scope='row')[35].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_may = float(soup.find_all(scope='row')[35].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_jun = float(soup.find_all(scope='row')[35].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_jul = float(soup.find_all(scope='row')[35].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_aug = float(soup.find_all(scope='row')[35].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_sep = float(soup.find_all(scope='row')[35].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_oct = float(soup.find_all(scope='row')[35].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_nov = float(soup.find_all(scope='row')[35].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
mean_min_dec = float(soup.find_all(scope='row')[35].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))

In [902]:
mean_min = [mean_min_jan,mean_min_feb,mean_min_march,mean_min_april,mean_min_may,mean_min_jun,mean_min_jul,mean_min_aug,mean_min_sep,
              mean_min_oct,mean_min_nov, mean_min_dec ]

mean_min 

[-12.0, -9.5, -5.8, -2.6, 1.7, 6.3, 8.9, 8.1, 3.9, -1.3, -5.0, -8.9]

In [903]:
record_low_jan = float(soup.find_all(scope='row')[36].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_feb = float(soup.find_all(scope='row')[36].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_march = float(soup.find_all(scope='row')[36].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_april = float(soup.find_all(scope='row')[36].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_may = float(soup.find_all(scope='row')[36].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_jun = float(soup.find_all(scope='row')[36].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_jul = float(soup.find_all(scope='row')[36].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_aug = float(soup.find_all(scope='row')[36].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_sep = float(soup.find_all(scope='row')[36].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_oct = float(soup.find_all(scope='row')[36].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_nov = float(soup.find_all(scope='row')[36].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_dec = float(soup.find_all(scope='row')[36].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))

In [904]:
record_low = [record_low_jan,record_low_feb,record_low_march,record_low_april,record_low_may,record_low_jun,record_low_jul,record_low_aug,record_low_sep,
              record_low_oct,record_low_nov, record_low_dec ]

record_low

[-25.3, -22.0, -19.1, -7.4, -2.8, 1.3, 4.9, 4.6, -0.9, -7.7, -17.8, -24.0]

In [905]:
avg_prep_jan = float(soup.find_all(scope='row')[37].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_feb = float(soup.find_all(scope='row')[37].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_march = float(soup.find_all(scope='row')[37].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_april = float(soup.find_all(scope='row')[37].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_may = float(soup.find_all(scope='row')[37].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_jun = float(soup.find_all(scope='row')[37].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_jul = float(soup.find_all(scope='row')[37].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_aug = float(soup.find_all(scope='row')[37].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_sep = float(soup.find_all(scope='row')[37].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_oct = float(soup.find_all(scope='row')[37].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_nov = float(soup.find_all(scope='row')[37].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_dec = float(soup.find_all(scope='row')[37].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [906]:
avg_prep = [avg_prep_jan,avg_prep_feb,avg_prep_march,avg_prep_april,avg_prep_may,avg_prep_jun,avg_prep_jul,avg_prep_aug,avg_prep_sep,
              avg_prep_oct,avg_prep_nov, avg_prep_dec ]

avg_prep

[41.5, 30.0, 35.9, 27.7, 52.8, 60.2, 70.0, 52.4, 43.6, 40.3, 38.8, 39.1]

In [907]:
avg_prep_days_jan = float(soup.find_all(scope='row')[38].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_feb = float(soup.find_all(scope='row')[38].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_march = float(soup.find_all(scope='row')[38].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_april = float(soup.find_all(scope='row')[38].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_may = float(soup.find_all(scope='row')[38].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_jun = float(soup.find_all(scope='row')[38].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_jul = float(soup.find_all(scope='row')[38].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_aug = float(soup.find_all(scope='row')[38].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_sep = float(soup.find_all(scope='row')[38].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_oct = float(soup.find_all(scope='row')[38].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_nov = float(soup.find_all(scope='row')[38].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_dec = float(soup.find_all(scope='row')[38].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [908]:
avg_prep_days = [avg_prep_days_jan,avg_prep_days_feb,avg_prep_days_march,avg_prep_days_april,avg_prep_days_may,avg_prep_days_jun,avg_prep_days_jul,avg_prep_days_aug,avg_prep_days_sep,
              avg_prep_days_oct,avg_prep_days_nov, avg_prep_days_dec ]

avg_prep_days

[15.8, 13.9, 14.0, 10.9, 12.8, 12.4, 13.4, 12.7, 11.6, 13.6, 14.5, 16.4]

In [909]:
avg_snowy_days_jan = float(soup.find_all(scope='row')[39].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_feb = float(soup.find_all(scope='row')[39].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_march = float(soup.find_all(scope='row')[39].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_april = float(soup.find_all(scope='row')[39].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_may = float(soup.find_all(scope='row')[39].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_jun = float(soup.find_all(scope='row')[39].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_jul = float(soup.find_all(scope='row')[39].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_aug = float(soup.find_all(scope='row')[39].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_sep = float(soup.find_all(scope='row')[39].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_oct = float(soup.find_all(scope='row')[39].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_nov = float(soup.find_all(scope='row')[39].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_dec = float(soup.find_all(scope='row')[39].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [910]:
avg_snowy_days = [avg_snowy_days_jan,avg_snowy_days_feb,avg_snowy_days_march,avg_snowy_days_april,avg_snowy_days_may,avg_snowy_days_jun,avg_snowy_days_jul,avg_snowy_days_aug,avg_snowy_days_sep,
              avg_snowy_days_oct,avg_snowy_days_nov, avg_snowy_days_dec ]

avg_snowy_days

[8.4, 6.8, 2.6, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4, 4.9]

In [911]:
avg_humidity_jan = float(soup.find_all(scope='row')[40].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_feb = float(soup.find_all(scope='row')[40].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_march = float(soup.find_all(scope='row')[40].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_april = float(soup.find_all(scope='row')[40].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_may = float(soup.find_all(scope='row')[40].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_jun = float(soup.find_all(scope='row')[40].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_jul = float(soup.find_all(scope='row')[40].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_aug = float(soup.find_all(scope='row')[40].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_sep = float(soup.find_all(scope='row')[40].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_oct = float(soup.find_all(scope='row')[40].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_nov = float(soup.find_all(scope='row')[40].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_dec = float(soup.find_all(scope='row')[40].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [912]:
avg_humidity = [avg_humidity_jan,avg_humidity_feb,avg_humidity_march,avg_humidity_april,avg_humidity_may,avg_humidity_jun,avg_humidity_jul,avg_humidity_aug,avg_humidity_sep,
              avg_humidity_oct,avg_humidity_nov, avg_humidity_dec ]

avg_humidity

[85.9, 81.2, 75.8, 67.2, 66.9, 66.3, 67.0, 68.5, 76.0, 82.7, 87.8, 87.5]

In [913]:
weather_add_info_berlin_df = pd.DataFrame([record_high, record_low, avg_prep, avg_prep_days, avg_snowy_days, avg_humidity], columns=['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jun',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec'])
weather_add_info_berlin_df

,Jan,Feb,Mar,Apr,May,Jun,Jun,Aug,Sep,Oct,Nov,Dec
0,15.1,19.2,25.8,30.8,32.7,38.4,38.3,38.0,32.3,27.7,20.9,15.6
1,-25.3,-22.0,-19.1,-7.4,-2.8,1.3,4.9,4.6,-0.9,-7.7,-17.8,-24.0
2,41.5,30.0,35.9,27.7,52.8,60.2,70.0,52.4,43.6,40.3,38.8,39.1
3,15.8,13.9,14.0,10.9,12.8,12.4,13.4,12.7,11.6,13.6,14.5,16.4
4,8.4,6.8,2.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.4,4.9
5,85.9,81.2,75.8,67.2,66.9,66.3,67.0,68.5,76.0,82.7,87.8,87.5


In [914]:
weather_add_info_berlin_df = weather_add_info_berlin_df.rename(index={0: 'record_high',
                                  1: 'record_low',
                                  2: 'avg_prep',
                                  3: 'avg_prep_days',
                                  4: 'avg_snowy_days',
                                  5: 'avg_humidity'})
                                 

weather_add_info_berlin_df

,Jan,Feb,Mar,Apr,May,Jun,Jun,Aug,Sep,Oct,Nov,Dec
record_high,15.1,19.2,25.8,30.8,32.7,38.4,38.3,38.0,32.3,27.7,20.9,15.6
record_low,-25.3,-22.0,-19.1,-7.4,-2.8,1.3,4.9,4.6,-0.9,-7.7,-17.8,-24.0
avg_prep,41.5,30.0,35.9,27.7,52.8,60.2,70.0,52.4,43.6,40.3,38.8,39.1
avg_prep_days,15.8,13.9,14.0,10.9,12.8,12.4,13.4,12.7,11.6,13.6,14.5,16.4
avg_snowy_days,8.4,6.8,2.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.4,4.9
avg_humidity,85.9,81.2,75.8,67.2,66.9,66.3,67.0,68.5,76.0,82.7,87.8,87.5


In [915]:
weather_add_info_berlin_df['city'] = 'Berlin'
weather_add_info_berlin_df

,Jan,Feb,Mar,Apr,May,Jun,Jun,Aug,Sep,Oct,Nov,Dec,city
record_high,15.1,19.2,25.8,30.8,32.7,38.4,38.3,38.0,32.3,27.7,20.9,15.6,Berlin
record_low,-25.3,-22.0,-19.1,-7.4,-2.8,1.3,4.9,4.6,-0.9,-7.7,-17.8,-24.0,Berlin
avg_prep,41.5,30.0,35.9,27.7,52.8,60.2,70.0,52.4,43.6,40.3,38.8,39.1,Berlin
avg_prep_days,15.8,13.9,14.0,10.9,12.8,12.4,13.4,12.7,11.6,13.6,14.5,16.4,Berlin
avg_snowy_days,8.4,6.8,2.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.4,4.9,Berlin
avg_humidity,85.9,81.2,75.8,67.2,66.9,66.3,67.0,68.5,76.0,82.7,87.8,87.5,Berlin


In [916]:
weather_add_info_berlin_df['timeline_data'] = ('1991 - 2020')
weather_add_info_berlin_df

,Jan,Feb,Mar,Apr,May,Jun,Jun,Aug,Sep,Oct,Nov,Dec,city,timeline_data
record_high,15.1,19.2,25.8,30.8,32.7,38.4,38.3,38.0,32.3,27.7,20.9,15.6,Berlin,1991 - 2020
record_low,-25.3,-22.0,-19.1,-7.4,-2.8,1.3,4.9,4.6,-0.9,-7.7,-17.8,-24.0,Berlin,1991 - 2020
avg_prep,41.5,30.0,35.9,27.7,52.8,60.2,70.0,52.4,43.6,40.3,38.8,39.1,Berlin,1991 - 2020
avg_prep_days,15.8,13.9,14.0,10.9,12.8,12.4,13.4,12.7,11.6,13.6,14.5,16.4,Berlin,1991 - 2020
avg_snowy_days,8.4,6.8,2.6,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1.4,4.9,Berlin,1991 - 2020
avg_humidity,85.9,81.2,75.8,67.2,66.9,66.3,67.0,68.5,76.0,82.7,87.8,87.5,Berlin,1991 - 2020


### Hamburg

In [918]:
url = 'https://en.wikipedia.org/wiki/Hamburg'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

In [919]:
soup.find_all(scope='row')[25]

<th scope="row" style="height: 16px;">Record high °C (°F)
</th>

In [920]:
record_high_jan = float(soup.find_all(scope='row')[25].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_feb = float(soup.find_all(scope='row')[25].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_march = float(soup.find_all(scope='row')[25].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_april = float(soup.find_all(scope='row')[25].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_may = float(soup.find_all(scope='row')[25].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_jun = float(soup.find_all(scope='row')[25].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_jul = float(soup.find_all(scope='row')[25].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_aug = float(soup.find_all(scope='row')[25].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_sep = float(soup.find_all(scope='row')[25].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_oct = float(soup.find_all(scope='row')[25].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_nov = float(soup.find_all(scope='row')[25].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
record_high_dec = float(soup.find_all(scope='row')[25].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [921]:
record_high = [record_high_jan,record_high_feb,record_high_march,record_high_april,record_high_may,record_high_jun,record_high_jul,record_high_aug,record_high_sep,
              record_high_oct,record_high_nov, record_high_dec ]
record_high

[14.4, 17.2, 23.0, 29.7, 33.5, 34.6, 40.1, 37.3, 32.3, 26.1, 20.2, 15.7]

In [922]:
record_low_jan = float(soup.find_all(scope='row')[29].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_feb = float(soup.find_all(scope='row')[29].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_march = float(soup.find_all(scope='row')[29].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_april = float(soup.find_all(scope='row')[29].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_may = float(soup.find_all(scope='row')[29].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_jun = float(soup.find_all(scope='row')[29].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_jul = float(soup.find_all(scope='row')[29].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_aug = float(soup.find_all(scope='row')[29].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_sep = float(soup.find_all(scope='row')[29].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_oct = float(soup.find_all(scope='row')[29].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_nov = float(soup.find_all(scope='row')[29].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))
record_low_dec = float(soup.find_all(scope='row')[29].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'").replace('−', '-'))

In [923]:
record_low = [record_low_jan,record_low_feb,record_low_march,record_low_april,record_low_may,record_low_jun,record_low_jul,record_low_aug,record_low_sep,
              record_low_oct,record_low_nov, record_low_dec ]

record_low

[-22.8, -29.1, -15.3, -7.1, -5.0, 0.6, 3.4, 1.8, -1.2, -7.1, -15.4, -18.5]

In [924]:
avg_prep_jan = float(soup.find_all(scope='row')[30].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_feb = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_march = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_april = float(soup.find_all(scope='row')[30].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_may = float(soup.find_all(scope='row')[30].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_jun = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_jul = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_aug = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_sep = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_oct = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_nov = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_dec = float(soup.find_all(scope='row')[30].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [925]:
avg_prep = [avg_prep_jan,avg_prep_feb,avg_prep_march,avg_prep_april,avg_prep_may,avg_prep_jun,avg_prep_jul,avg_prep_aug,avg_prep_sep,
              avg_prep_oct,avg_prep_nov, avg_prep_dec ]

avg_prep

[66.7, 54.9, 56.7, 39.2, 57.8, 74.4, 81.8, 77.5, 64.7, 63.0, 61.1, 72.6]

In [926]:
avg_prep_days_jan = float(soup.find_all(scope='row')[31].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_feb = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_march = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_april = float(soup.find_all(scope='row')[31].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_may = float(soup.find_all(scope='row')[31].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_jun = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_jul = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_aug = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_sep = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_oct = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_nov = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_prep_days_dec = float(soup.find_all(scope='row')[31].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [927]:
avg_prep_days = [avg_prep_days_jan,avg_prep_days_feb,avg_prep_days_march,avg_prep_days_april,avg_prep_days_may,avg_prep_days_jun,avg_prep_days_jul,avg_prep_days_aug,avg_prep_days_sep,
              avg_prep_days_oct,avg_prep_days_nov, avg_prep_days_dec ]

avg_prep_days

[17.7, 16.2, 15.2, 12.8, 13.8, 15.3, 16.0, 15.8, 14.5, 16.2, 16.9, 18.0]

In [928]:
avg_snowy_days_jan = float(soup.find_all(scope='row')[32].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_feb = float(soup.find_all(scope='row')[32].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_march = float(soup.find_all(scope='row')[32].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_april = float(soup.find_all(scope='row')[32].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_may = float(soup.find_all(scope='row')[32].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_jun = float(soup.find_all(scope='row')[32].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_jul = float(soup.find_all(scope='row')[32].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_aug = float(soup.find_all(scope='row')[32].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_sep = float(soup.find_all(scope='row')[32].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_oct = float(soup.find_all(scope='row')[32].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_nov = float(soup.find_all(scope='row')[32].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_snowy_days_dec = float(soup.find_all(scope='row')[32].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [929]:
avg_snowy_days = [avg_snowy_days_jan,avg_snowy_days_feb,avg_snowy_days_march,avg_snowy_days_april,avg_snowy_days_may,avg_snowy_days_jun,avg_snowy_days_jul,avg_snowy_days_aug,avg_snowy_days_sep,
              avg_snowy_days_oct,avg_snowy_days_nov, avg_snowy_days_dec ]

avg_snowy_days

[5.9, 5.0, 2.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2, 3.5]

In [930]:
avg_humidity_jan = float(soup.find_all(scope='row')[33].find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_feb = float(soup.find_all(scope='row')[33].find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_march = float(soup.find_all(scope='row')[33].find_next('td').find_next('td').find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_april = float(soup.find_all(scope='row')[33].find_next('td').find_next('td')
                                    .find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_may = float(soup.find_all(scope='row')[33].find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_jun = float(soup.find_all(scope='row')[33].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_jul = float(soup.find_all(scope='row')[33].find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_aug = float(soup.find_all(scope='row')[33].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_sep = float(soup.find_all(scope='row')[33].find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_oct = float(soup.find_all(scope='row')[33].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_nov = float(soup.find_all(scope='row')[33].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))
avg_humidity_dec = float(soup.find_all(scope='row')[33].find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').find_next('td').find_next('td').find_next('td').find_next('td')
                                    .find_next('td').get_text().split('\n')[0].strip("'").split('(')[0].strip("'"))

In [931]:
avg_humidity = [avg_humidity_jan,avg_humidity_feb,avg_humidity_march,avg_humidity_april,avg_humidity_may,avg_humidity_jun,avg_humidity_jul,avg_humidity_aug,avg_humidity_sep,
              avg_humidity_oct,avg_humidity_nov, avg_humidity_dec ]

avg_humidity

[85.8, 82.6, 77.7, 71.0, 70.8, 72.1, 72.6, 74.3, 79.4, 83.4, 87.1, 87.6]

In [932]:
weather_add_info_hamburg_df = pd.DataFrame([record_high, record_low, avg_prep, avg_prep_days, avg_snowy_days, avg_humidity], columns=['Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun',
 'Jun',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec'])
weather_add_info_hamburg_df

,Jan,Feb,Mar,Apr,May,Jun,Jun,Aug,Sep,Oct,Nov,Dec
0,14.4,17.2,23.0,29.7,33.5,34.6,40.1,37.3,32.3,26.1,20.2,15.7
1,-22.8,-29.1,-15.3,-7.1,-5.0,0.6,3.4,1.8,-1.2,-7.1,-15.4,-18.5
2,66.7,54.9,56.7,39.2,57.8,74.4,81.8,77.5,64.7,63.0,61.1,72.6
3,17.7,16.2,15.2,12.8,13.8,15.3,16.0,15.8,14.5,16.2,16.9,18.0
4,5.9,5.0,2.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,3.5
5,85.8,82.6,77.7,71.0,70.8,72.1,72.6,74.3,79.4,83.4,87.1,87.6


In [933]:
weather_add_info_hamburg_df['city'] = 'Hamburg'
weather_add_info_hamburg_df

,Jan,Feb,Mar,Apr,May,Jun,Jun,Aug,Sep,Oct,Nov,Dec,city
0,14.4,17.2,23.0,29.7,33.5,34.6,40.1,37.3,32.3,26.1,20.2,15.7,Hamburg
1,-22.8,-29.1,-15.3,-7.1,-5.0,0.6,3.4,1.8,-1.2,-7.1,-15.4,-18.5,Hamburg
2,66.7,54.9,56.7,39.2,57.8,74.4,81.8,77.5,64.7,63.0,61.1,72.6,Hamburg
3,17.7,16.2,15.2,12.8,13.8,15.3,16.0,15.8,14.5,16.2,16.9,18.0,Hamburg
4,5.9,5.0,2.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,3.5,Hamburg
5,85.8,82.6,77.7,71.0,70.8,72.1,72.6,74.3,79.4,83.4,87.1,87.6,Hamburg


In [934]:
weather_add_info_hamburg_df = weather_add_info_hamburg_df.rename(index={0: 'record_high',
                                  1: 'record_low',
                                  2: 'avg_prep',
                                  3: 'avg_prep_days',
                                  4: 'avg_snowy_days',
                                  5: 'avg_humidity'})
                                 

weather_add_info_hamburg_df

,Jan,Feb,Mar,Apr,May,Jun,Jun,Aug,Sep,Oct,Nov,Dec,city
record_high,14.4,17.2,23.0,29.7,33.5,34.6,40.1,37.3,32.3,26.1,20.2,15.7,Hamburg
record_low,-22.8,-29.1,-15.3,-7.1,-5.0,0.6,3.4,1.8,-1.2,-7.1,-15.4,-18.5,Hamburg
avg_prep,66.7,54.9,56.7,39.2,57.8,74.4,81.8,77.5,64.7,63.0,61.1,72.6,Hamburg
avg_prep_days,17.7,16.2,15.2,12.8,13.8,15.3,16.0,15.8,14.5,16.2,16.9,18.0,Hamburg
avg_snowy_days,5.9,5.0,2.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,3.5,Hamburg
avg_humidity,85.8,82.6,77.7,71.0,70.8,72.1,72.6,74.3,79.4,83.4,87.1,87.6,Hamburg


In [935]:
weather_add_info_hamburg_df['timeline_data'] = ('1991 - 2020')
weather_add_info_hamburg_df

,Jan,Feb,Mar,Apr,May,Jun,Jun,Aug,Sep,Oct,Nov,Dec,city,timeline_data
record_high,14.4,17.2,23.0,29.7,33.5,34.6,40.1,37.3,32.3,26.1,20.2,15.7,Hamburg,1991 - 2020
record_low,-22.8,-29.1,-15.3,-7.1,-5.0,0.6,3.4,1.8,-1.2,-7.1,-15.4,-18.5,Hamburg,1991 - 2020
avg_prep,66.7,54.9,56.7,39.2,57.8,74.4,81.8,77.5,64.7,63.0,61.1,72.6,Hamburg,1991 - 2020
avg_prep_days,17.7,16.2,15.2,12.8,13.8,15.3,16.0,15.8,14.5,16.2,16.9,18.0,Hamburg,1991 - 2020
avg_snowy_days,5.9,5.0,2.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,3.5,Hamburg,1991 - 2020
avg_humidity,85.8,82.6,77.7,71.0,70.8,72.1,72.6,74.3,79.4,83.4,87.1,87.6,Hamburg,1991 - 2020


In [936]:
weather_add_info_hamburg_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, record_high to avg_humidity
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Jan            6 non-null      float64
 1   Feb            6 non-null      float64
 2   Mar            6 non-null      float64
 3   Apr            6 non-null      float64
 4   May            6 non-null      float64
 5   Jun            6 non-null      float64
 6   Jun            6 non-null      float64
 7   Aug            6 non-null      float64
 8   Sep            6 non-null      float64
 9   Oct            6 non-null      float64
 10  Nov            6 non-null      float64
 11  Dec            6 non-null      float64
 12  city           6 non-null      object 
 13  timeline_data  6 non-null      object 
dtypes: float64(12), object(2)
memory usage: 720.0+ bytes


In [978]:
data = {'Berlin':  {'Record_high (°C)': [15.1, 19.2, 25.8, 30.8, 32.7, 38.4, 38.3, 38.0, 32.3, 27.7, 20.9, 15.6],                
                   'Record_low': [-25.3, -22.0, -19.1, -7.4, -2.8, 1.3, 4.9, 4.6, -0.9, -7.7, -17.8, -24.0],
                   'Avg_prep_mm': [41.5, 30.0, 35.9, 27.7, 52.8, 60.2, 70.0, 52.4, 43.6, 40.3, 38.8, 39.1],
                   'Avg_prep_days':[15.8, 13.9, 14.0, 10.9, 12.8, 12.4, 13.4, 12.7, 11.6, 13.6, 14.5, 16.4],
                   'Avg_snowy_days':[8.4, 6.8, 2.6, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4, 4.9],
                   'Avg_humidity':[85.9, 81.2, 75.8, 67.2, 66.9, 66.3, 67.0, 68.5, 76.0, 82.7, 87.8, 87.5]},
        'Hamburg': {'Record_high (°C)':[14.4, 17.2, 23.0, 29.7, 33.5, 34.6, 40.1, 37.3, 32.3, 26.1, 20.2, 15.7],
                   'Record_low':[-22.8, -29.1, -15.3, -7.1, -5.0, 0.6, 3.4, 1.8, -1.2, -7.1, -15.4, -18.5],
                   'Avg_prep_mm':[66.7, 54.9, 56.7, 39.2, 57.8, 74.4, 81.8, 77.5, 64.7, 63.0, 61.1, 72.6],
                   'Avg_prep_days':[17.7, 16.2, 15.2, 12.8, 13.8, 15.3, 16.0, 15.8, 14.5, 16.2, 16.9, 18.0],
                   'Avg_snowy_days':[5.9, 5.0, 2.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.2, 3.5],
                   'Avg_humidity':[85.8, 82.6, 77.7, 71.0, 70.8, 72.1, 72.6, 74.3, 79.4, 83.4, 87.1, 87.6]},   
        'Munich':  {'Record_high (°C)':[18.9, 21.4, 24.0, 32.2, 31.8, 35.2, 37.5, 37.0, 31.8, 28.2, 24.2, 21.7],
                   'Record_low':[-22.2, -25.4, -16.0, -6.0, -2.3, 1.0, 6.5, 4.8, 0.6, -4.5, -11.0, -20.7],
                   'Avg_prep_mm':[51.9, 45.5, 61.2, 56.0, 107.0, 120.9, 118.9, 116.5, 78.1, 66.9, 58.4, 58.5],
                   'Avg_prep_days':[15.3, 14.0, 15.6, 13.5, 16.1, 16.7, 16.1, 15.0, 14.2, 14.2, 14.6, 16.8],
                   'Avg_snowy_days':[11.7, 11.2, 4.5, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.3, 8.0],
                   'Avg_humidity':[80.3, 75.9, 70.7, 64.6, 67.2, 66.1, 68.1, 75.5, 79.9, 83.3, 82.3]} 
        }

cities_weather_fun_facts_df = pd.DataFrame.from_dict(data)
cities_weather_fun_facts_df

,Berlin,Hamburg,Munich
Record_high (°C),"[15.1, 19.2, 25.8, 30.8, 32.7, 38.4, 38.3, 38....","[14.4, 17.2, 23.0, 29.7, 33.5, 34.6, 40.1, 37....","[18.9, 21.4, 24.0, 32.2, 31.8, 35.2, 37.5, 37...."
Record_low,"[-25.3, -22.0, -19.1, -7.4, -2.8, 1.3, 4.9, 4....","[-22.8, -29.1, -15.3, -7.1, -5.0, 0.6, 3.4, 1....","[-22.2, -25.4, -16.0, -6.0, -2.3, 1.0, 6.5, 4...."
Avg_prep_mm,"[41.5, 30.0, 35.9, 27.7, 52.8, 60.2, 70.0, 52....","[66.7, 54.9, 56.7, 39.2, 57.8, 74.4, 81.8, 77....","[51.9, 45.5, 61.2, 56.0, 107.0, 120.9, 118.9, ..."
Avg_prep_days,"[15.8, 13.9, 14.0, 10.9, 12.8, 12.4, 13.4, 12....","[17.7, 16.2, 15.2, 12.8, 13.8, 15.3, 16.0, 15....","[15.3, 14.0, 15.6, 13.5, 16.1, 16.7, 16.1, 15...."
Avg_snowy_days,"[8.4, 6.8, 2.6, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[5.9, 5.0, 2.9, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[11.7, 11.2, 4.5, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0..."
Avg_humidity,"[85.9, 81.2, 75.8, 67.2, 66.9, 66.3, 67.0, 68....","[85.8, 82.6, 77.7, 71.0, 70.8, 72.1, 72.6, 74....","[80.3, 75.9, 70.7, 64.6, 67.2, 66.1, 68.1, 75...."


## Weather_add_info to.sql()

## Flights_info

In [939]:
url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

querystring = {"lat":"52.52","lon":"13.405","radiusKm":"30","limit":"5","withFlightInfoOnly":"true"}

headers = {
	"x-rapidapi-key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

print(response.json())

{'searchBy': {'lat': 52.52, 'lon': 13.405}, 'count': 1, 'items': [{'icao': 'EDDB', 'iata': 'BER', 'name': 'Berlin Brandenburg', 'shortName': 'Brandenburg', 'municipalityName': 'Berlin', 'location': {'lat': 52.35139, 'lon': 13.493889}, 'countryCode': 'DE', 'timeZone': 'Europe/Berlin'}]}


In [940]:
response.json()['items'][0].update({'city_id': 1})

In [941]:
pd.json_normalize(response.json()['items'])

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,52.35139,13.493889


In [942]:
def icao_airport_codes(df):
    list_for_df = []
    for index, row in df.iterrows():
        url = "https://aerodatabox.p.rapidapi.com/airports/search/location"
        
        querystring = {"lat":str(row['latitude']),"lon":str(row['longitude']),"radiusKm":"30","limit":"5","withFlightInfoOnly":"true"}
        
        headers = {
        	"x-rapidapi-key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
        	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            response_json = response.json()
            response_json['items'][0].update({'city_id': row['city_id']})
            list_for_df.append(pd.json_normalize(response_json['items']))

    return pd.concat(list_for_df, ignore_index=True)

In [943]:
airports_df = icao_airport_codes(cities_stable_from_sql)
airports_df 

,icao,iata,name,shortName,municipalityName,countryCode,timeZone,city_id,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,1,52.35139,13.493889
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,2,53.63040,9.988229
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,3,48.35380,11.786100


In [944]:
airports_df = airports_df.rename(columns={'name': 'a_name'})
airports_df

,icao,iata,a_name,shortName,municipalityName,countryCode,timeZone,city_id,location.lat,location.lon
0,EDDB,BER,Berlin Brandenburg,Brandenburg,Berlin,DE,Europe/Berlin,1,52.35139,13.493889
1,EDDH,HAM,Hamburg,Hamburg,Hamburg,DE,Europe/Berlin,2,53.63040,9.988229
2,EDDM,MUC,Munich,Munich,Munich,DE,Europe/Berlin,3,48.35380,11.786100


In [945]:
airports_df = airports_df[['icao', 'a_name', 'city_id']]
airports_df

,icao,a_name,city_id
0,EDDB,Berlin Brandenburg,1
1,EDDH,Hamburg,2
2,EDDM,Munich,3


### Airports_df to.sql()

In [981]:
#airports_df.to_sql(
    #'cities_airports',
    #if_exists='append',
    #con=connection_string,
    #index=False)

3

In [983]:
cities_airports_from_sql = pd.read_sql("cities_airports", con=connection_string)
cities_airports_from_sql

,icao,a_name,city_id
0,EDDB,Berlin Brandenburg,1
1,EDDH,Hamburg,2
2,EDDM,Munich,3


### Flights

In [985]:
cities_airports_from_sql

,icao,a_name,city_id
0,EDDB,Berlin Brandenburg,1
1,EDDH,Hamburg,2
2,EDDM,Munich,3


In [951]:
url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{EDDB}/{2024-08-13}T{12}:00/{2024-08-13}T{23}:59"

querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

headers = {
	"x-rapidapi-key": "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4",
	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (2248233815.py, line 1)

In [989]:
(pd.Timestamp.now()+pd.Timedelta(24, 'hours')).strftime("%Y-%m-%d")

'2024-08-14'

In [990]:
times = [['00', '11'], ['12', '23']]

In [991]:
API_key = "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4"
tomorrow = (pd.Timestamp.now() + pd.Timedelta(24, 'hours')).strftime('%Y-%m-%d')
times = [['00', '11'], ['12', '23']]
for index, row in airports_df.iloc[0:1].iterrows():
    for time in times[0:1]:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['icao']}/{tomorrow}T{time[0]}:00/{tomorrow}T{time[1]}:59"
        
        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}
        
        headers = {
        	"x-rapidapi-key": API_key,
        	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }
        
        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            response_json = response.json()
        else:
            print(f'Error at {row["icao"]}: {response.message}')

In [992]:
response.json()

{'arrivals': [{'movement': {'airport': {'icao': 'LTBJ',
     'iata': 'ADB',
     'name': 'İzmir',
     'timeZone': 'Europe/Istanbul'},
    'scheduledTime': {'utc': '2024-08-14 04:30Z',
     'local': '2024-08-14 06:30+02:00'},
    'terminal': '1',
    'quality': ['Basic']},
   'number': 'XQ 966',
   'status': 'Expected',
   'codeshareStatus': 'Unknown',
   'isCargo': False,
   'aircraft': {'model': 'Boeing 737-800'},
   'airline': {'name': 'Sun Express', 'iata': 'XQ', 'icao': 'SXS'}},
  {'movement': {'airport': {'icao': 'ZBAA',
     'iata': 'PEK',
     'name': 'Beijing',
     'timeZone': 'Asia/Shanghai'},
    'scheduledTime': {'utc': '2024-08-14 04:45Z',
     'local': '2024-08-14 06:45+02:00'},
    'terminal': '1',
    'quality': ['Basic']},
   'number': 'HU 489',
   'status': 'Expected',
   'codeshareStatus': 'IsOperator',
   'isCargo': False,
   'aircraft': {'model': 'Airbus A330-300'},
   'airline': {'name': 'Hainan', 'iata': 'HU', 'icao': 'CHH'}},
  {'movement': {'airport': {'icao':

In [993]:
len(response_json['arrivals'])

67

In [994]:
response_json['arrivals'][0]['movement']['airport']['name']
response_json['arrivals'][0]['movement']['airport']['icao']
response_json['arrivals'][0]['movement'].get('revisedTime', response_json['arrivals'][0]['movement']['scheduledTime'])['local']
response_json['arrivals'][0]['aircraft']['model']
response_json['arrivals'][0]['number']

'XQ 966'

In [995]:
flight_dict = {'arrival_icao': [],
               'departing': [],
               'departure_icao': [],
               'arrival_time': [],
               'model': [],
               'flight_num': []
              }
for flight in response_json['arrivals']:
    flight_dict['arrival_icao'].append(row['icao'])
    flight_dict['departing'].append(flight['movement']['airport']['name'])
    flight_dict['departure_icao'].append(flight['movement']['airport'].get('icao'))
    flight_dict['arrival_time'].append(flight['movement'].get('revisedTime', flight['movement']['scheduledTime'])['local'])
    flight_dict['model'].append(flight['aircraft']['model'])
    flight_dict['flight_num'].append(flight['number'])
flights_df = pd.DataFrame(flight_dict)
flights_df['arrival_time'] = pd.to_datetime(flights_df['arrival_time'])

In [996]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype                    
---  ------          --------------  -----                    
 0   arrival_icao    67 non-null     object                   
 1   departing       67 non-null     object                   
 2   departure_icao  66 non-null     object                   
 3   arrival_time    67 non-null     datetime64[ns, UTC+02:00]
 4   model           67 non-null     object                   
 5   flight_num      67 non-null     object                   
dtypes: datetime64[ns, UTC+02:00](1), object(5)
memory usage: 3.3+ KB


In [997]:
API_key = "14a44098c8mshe4536a007985112p1e3b4bjsn8fd805eb6bd4"
tomorrow = (pd.Timestamp.now() + pd.Timedelta(24, 'hours')).strftime('%Y-%m-%d')
times = [['00', '11'], ['12', '23']]
flight_dict = {'arrival_icao': [],
               'departing': [],
               'departure_icao': [],
               'arrival_time': [],
               'model': [],
               'flight_num': []
              }
for index, row in airports_df.iterrows():
    for time in times:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{row['icao']}/{tomorrow}T{time[0]}:00/{tomorrow}T{time[1]}:59"

        querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"true","withCargo":"false","withPrivate":"false","withLocation":"false"}

        headers = {
        	"x-rapidapi-key": API_key,
        	"x-rapidapi-host": "aerodatabox.p.rapidapi.com"
        }

        response = requests.get(url, headers=headers, params=querystring)
        if response.status_code == 200:
            response_json = response.json()
            for flight in response_json['arrivals']:
                flight_dict['arrival_icao'].append(row['icao'])
                flight_dict['departing'].append(flight['movement']['airport'].get('name'))
                flight_dict['departure_icao'].append(flight['movement']['airport'].get('icao'))
                flight_dict['arrival_time'].append(flight['movement'].get('revisedTime', flight['movement']['scheduledTime'])['local'])
                flight_dict['model'].append(flight.get('aircraft',{}).get('model'))
                flight_dict['flight_num'].append(flight['number'])
        else:
            print(f'Error at {row["icao"]}: {response.message}')
flights_df = pd.DataFrame(flight_dict)
flights_df['arrival_time'] = pd.to_datetime(flights_df['arrival_time'])
flights_df

,arrival_icao,departing,departure_icao,arrival_time,model,flight_num
0,EDDB,İzmir,LTBJ,2024-08-14 06:30:00+02:00,Boeing 737-800,XQ 966
1,EDDB,Beijing,ZBAA,2024-08-14 06:45:00+02:00,Airbus A330-300,HU 489
2,EDDB,Samsun,LTFH,2024-08-14 06:45:00+02:00,Boeing 737-800,XQ 1768
3,EDDB,Newark,KEWR,2024-08-14 07:15:00+02:00,Boeing 767-400,UA 962
4,EDDB,Doha,OTHH,2024-08-14 07:20:00+02:00,Boeing 787-9,QR 79
...,...,...,...,...,...,...
888,EDDM,Gran Canaria Island,GCLP,2024-08-14 23:25:00+02:00,Boeing 737-800,X3 2239
889,EDDM,Lisbon,LPPT,2024-08-14 23:30:00+02:00,Airbus A320 NEO,TP 556
890,EDDM,Hamburg,EDDH,2024-08-14 22:30:00+02:00,Airbus A319,LH 2071
891,EDDM,Amsterdam,EHAM,2024-08-14 22:30:00+02:00,Airbus A320,LH 2309


In [998]:
flights_df = flights_df.merge(cities_airports_from_sql,
                                left_on='arrival_icao',
                                right_on='icao')

flights_df

,arrival_icao,departing,departure_icao,arrival_time,model,flight_num,icao,a_name,city_id
0,EDDB,İzmir,LTBJ,2024-08-14 06:30:00+02:00,Boeing 737-800,XQ 966,EDDB,Berlin Brandenburg,1
1,EDDB,Beijing,ZBAA,2024-08-14 06:45:00+02:00,Airbus A330-300,HU 489,EDDB,Berlin Brandenburg,1
2,EDDB,Samsun,LTFH,2024-08-14 06:45:00+02:00,Boeing 737-800,XQ 1768,EDDB,Berlin Brandenburg,1
3,EDDB,Newark,KEWR,2024-08-14 07:15:00+02:00,Boeing 767-400,UA 962,EDDB,Berlin Brandenburg,1
4,EDDB,Doha,OTHH,2024-08-14 07:20:00+02:00,Boeing 787-9,QR 79,EDDB,Berlin Brandenburg,1
...,...,...,...,...,...,...,...,...,...
888,EDDM,Gran Canaria Island,GCLP,2024-08-14 23:25:00+02:00,Boeing 737-800,X3 2239,EDDM,Munich,3
889,EDDM,Lisbon,LPPT,2024-08-14 23:30:00+02:00,Airbus A320 NEO,TP 556,EDDM,Munich,3
890,EDDM,Hamburg,EDDH,2024-08-14 22:30:00+02:00,Airbus A319,LH 2071,EDDM,Munich,3
891,EDDM,Amsterdam,EHAM,2024-08-14 22:30:00+02:00,Airbus A320,LH 2309,EDDM,Munich,3


In [999]:
flights_df = flights_df.drop(columns=['icao', 'a_name', 'city_id'])
flights_df

,arrival_icao,departing,departure_icao,arrival_time,model,flight_num
0,EDDB,İzmir,LTBJ,2024-08-14 06:30:00+02:00,Boeing 737-800,XQ 966
1,EDDB,Beijing,ZBAA,2024-08-14 06:45:00+02:00,Airbus A330-300,HU 489
2,EDDB,Samsun,LTFH,2024-08-14 06:45:00+02:00,Boeing 737-800,XQ 1768
3,EDDB,Newark,KEWR,2024-08-14 07:15:00+02:00,Boeing 767-400,UA 962
4,EDDB,Doha,OTHH,2024-08-14 07:20:00+02:00,Boeing 787-9,QR 79
...,...,...,...,...,...,...
888,EDDM,Gran Canaria Island,GCLP,2024-08-14 23:25:00+02:00,Boeing 737-800,X3 2239
889,EDDM,Lisbon,LPPT,2024-08-14 23:30:00+02:00,Airbus A320 NEO,TP 556
890,EDDM,Hamburg,EDDH,2024-08-14 22:30:00+02:00,Airbus A319,LH 2071
891,EDDM,Amsterdam,EHAM,2024-08-14 22:30:00+02:00,Airbus A320,LH 2309


### Flights_df to.sql()

In [1001]:
#flights_df.to_sql(
               # 'flights',
                #if_exists='append',
                #con=connection_string,
                #index=False)

893

In [1002]:
flights_from_sql = pd.read_sql("flights", con=connection_string)
flights_from_sql

,flight_id,arrival_icao,departing,departure_icao,arrival_time,model,flight_num
0,1,EDDB,İzmir,LTBJ,2024-08-14,Boeing 737-800,XQ 966
1,2,EDDB,Beijing,ZBAA,2024-08-14,Airbus A330-300,HU 489
2,3,EDDB,Samsun,LTFH,2024-08-14,Boeing 737-800,XQ 1768
3,4,EDDB,Newark,KEWR,2024-08-14,Boeing 767-400,UA 962
4,5,EDDB,Doha,OTHH,2024-08-14,Boeing 787-9,QR 79
...,...,...,...,...,...,...,...
888,889,EDDM,Gran Canaria Island,GCLP,2024-08-14,Boeing 737-800,X3 2239
889,890,EDDM,Lisbon,LPPT,2024-08-14,Airbus A320 NEO,TP 556
890,891,EDDM,Hamburg,EDDH,2024-08-14,Airbus A319,LH 2071
891,892,EDDM,Amsterdam,EHAM,2024-08-14,Airbus A320,LH 2309
